## Logistic regression module of pyspark is ingaged in this project

This project helps me to explore the Databricks environment for pyspark. Also, I implimented the logistic regression module to predict the servivers in the Titanic accident. The accuracy is 75%, the code requires some fine tuning but it serves the purpose.

In [0]:
df = spark.read.format("csv").load("dbfs:/FileStore/shared_uploads/johnny.getachew@gmail.com/titanic.csv",header=True,inferSchema=True)

In [0]:
df.printSchema()

root
-- PassengerId: integer (nullable = true)
-- Survived: integer (nullable = true)
-- Pclass: integer (nullable = true)
-- Name: string (nullable = true)
-- Sex: string (nullable = true)
-- Age: double (nullable = true)
-- SibSp: integer (nullable = true)
-- Parch: integer (nullable = true)
-- Ticket: string (nullable = true)
-- Fare: double (nullable = true)
-- Cabin: string (nullable = true)
-- Embarked: string (nullable = true)

In [0]:
df.columns

Out[4]: ['PassengerId',
 'Survived',
 'Pclass',
 'Name',
 'Sex',
 'Age',
 'SibSp',
 'Parch',
 'Ticket',
 'Fare',
 'Cabin',
 'Embarked']

In [0]:
my_cols = df.select(['Survived', 'Pclass','Sex', 'Age', 'SibSp', 'Parch','Fare', 'Embarked'])

In [0]:
my_final_df = my_cols.na.drop()

In [0]:
from pyspark.ml.feature import VectorAssembler, VectorIndexer,OneHotEncoder,StringIndexer

In [0]:
gender_ind = StringIndexer(inputCol='Sex',outputCol='SexIndex')
gender_incoder = OneHotEncoder(inputCol='SexIndex',outputCol='SexVec')

In [0]:
embark_ind = StringIndexer(inputCol='Embarked',outputCol='EmbarkIndex')
embark_incoder = OneHotEncoder(inputCol='EmbarkIndex',outputCol='EmbarkVec')

In [0]:
assembler = VectorAssembler(inputCols=['Pclass','SexVec','Age','SibSp','Parch','Fare','EmbarkVec'],outputCol='features')

In [0]:
from pyspark.ml.classification import LogisticRegression
from pyspark.ml import Pipeline

In [0]:
log_reg_tit = LogisticRegression(featuresCol='features',labelCol='Survived')

In [0]:
pipeline = Pipeline(stages=[gender_ind,embark_ind,gender_incoder,embark_incoder,assembler,log_reg_tit])

In [0]:
train_dt,test_dt = my_final_df.randomSplit([0.70,0.30])

In [0]:
fit_model = pipeline.fit(train_dt)

In [0]:
results = fit_model.transform(test_dt)

In [0]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [0]:
my_eval = BinaryClassificationEvaluator(rawPredictionCol='prediction',labelCol='Survived')

In [0]:
results.select('Survived','prediction').show()

+--------+----------+
Survived|prediction|
+--------+----------+
 0| 1.0|
 0| 1.0|
 0| 1.0|
 0| 1.0|
 0| 1.0|
 0| 1.0|
 0| 0.0|
 0| 0.0|
 0| 0.0|
 0| 0.0|
 0| 0.0|
 0| 0.0|
 0| 0.0|
 0| 0.0|
 0| 0.0|
 0| 0.0|
 0| 0.0|
 0| 0.0|
 0| 0.0|
 0| 0.0|
+--------+----------+
only showing top 20 rows

In [0]:
AUC = my_eval.evaluate(results)

In [0]:
AUC

Out[21]: 0.75